# Demo: Estimating Option Price Using Amplitude Estimation

## Introduction and background

#

In Finance models we are often interested in calculating the average of a function of a given probability distribution ($E[f(x)]$). The most popular method to estimate the average is Monte Carlo [[1](#MCMF)] due to its flexibility and ability to generically handle stochastic parameters.
Classical Monte Carlo methods, however, generally require extensive computational resources to provide an accurate estimation.
By leveraging the laws of quantum mechanics, a quantum computer may provide novel ways to
solve computationally intensive financial problems, such as risk management, portfolio optimization, and option pricing.
The core quantum advantage of several of these applications is the Amplitude Estimation algorithm [[2](#AEA)] which can estimate a parameter with a
convergence rate of $\Omega(1/M^{1/2})$, compared to $\Omega(1/M)$ in the classical case, where $M$ is the number of Grover iterations in the quantum case and the number of the Monte Carlo samples in the classical case. This represents a theoretical quadratic speed-up of the quantum method over classical Monte Carlo methods.


### Option pricing

An option is the possibility to buy (call) or sell (put) an item (or share) at a known price - the strike price (K), where the option has a maturity price (S).
The payoff function to describe for example a European call option will be:

$f(S)=\
\Bigg\{\begin{array}{lr}
    0, & \text{when } K\geq S\\
    S - K, & \text{when } K < S\end{array}
$

The maturity price is unknown. Therefore, it is expressed by a price distribution function, which may be any type of a distribution function.
For example a log-normal distribution: $\mathcal{ln}(S)\sim~\mathcal{N}(\mu,\sigma)$,
where $\mathcal{N}(\mu,\sigma)$ is the standard normal distribution with mean equal to $\mu$ and sdt equal to $\sigma$ .

To estimate the average option price using a quantum computer, one needs to:

- Load the distribution, that is, discretize the distribution using $2^n$ points (n is the number of qubits) and truncate it.

- Implement the payoff function that is equal to zero if $S\leq{K}$ and increases linearly otherwise.
  The linear part is approximated in order to be loaded properly using $R_y$ rotations [[3](#QAR)].

- Evaluate the expected payoff using amplitude estimation


### The probability distribution

We begin by creating the probability distribution. We use `FinanceModelInput`, to build discrete version of the log normal probability with $2^n$ points, when $\mu$ is equal to `mu`, $\sigma$ is equal to `sigma` and $n$ is equal to `num_qubits`.

In [1]:
from classiq.applications.finance import log_normal_model_input, model_input

num_qubits = 5
mu = 0.7
sigma = 0.13

log_normal_model = log_normal_model_input.LogNormalModelInput(
    num_qubits=num_qubits, mu=mu, sigma=sigma
)

### The option pricing function

We continue with creating the option pricing function . We use `FinanceFunctionInput`, to build the European call option function, when $K$ equal to `threshold`.

In [2]:
from classiq.applications.finance import function_input

threshold = 1.9

condition = function_input.FunctionCondition(threshold=threshold, larger=True)
finance_function = function_input.FinanceFunctionInput(
    f="european call option",
    condition=condition,
)

### Loading the model

After defining the probability distribution and the finance function input, we proceed by creating a Model. The Model define the logic flow of the quantum algorithm. In this case, the logic flow includes one `Finance` operator that loads the distribution and then implements the payoff function, which is plugged into the amplitude estimation algorithm (using phase estimation). We need to define the number of qubits used by the phase estimation algorithm which will set the accuracy of the calculation.

In [3]:
from classiq import construct_finance_model

qmod = construct_finance_model(
    finance_model_input=log_normal_model,
    finance_function_input=finance_function,
    phase_port_size=2,
)

### Setting Constraints to the circuit

We set constraints for the number of qubits in order to get an optimized quantum circuit.

In [4]:
from classiq import Constraints
from classiq.synthesis import set_constraints

qmod = set_constraints(qmod, constraints=Constraints(max_width=20))

In [5]:
from classiq import write_qmod

write_qmod(qmod, "option_pricing")

### Synthesizing the circuit

Now we are ready to synthesize the circuit using Classiq's synthesis engine. The synthesis should take approximately several seconds:

In [6]:
from classiq import synthesize

qprog = synthesize(qmod)

### Showing the Resulting Circuit

After Classiq's synthesis engine has finished the job, we can show the resulting circuit in the interactive GUI:

In [7]:
from classiq import QuantumProgram, show

show(qprog)

Opening: https://platform.classiq.io/circuit/0e699543-3ee0-4359-b84e-60fb029bac41?version=0.38.0.dev42%2Bfd36e2c41c


In [8]:
circuit = QuantumProgram.from_qprog(qprog)
print(circuit.transpiled_circuit.depth)

2593


### Executing the circuit

Lastly, we can execute the resulting circuit with Classiq's execute interface, using the `execute` function.

In [9]:
from classiq import execute

results = execute(qprog).result()

Printing out the result estimation of the options price :

In [10]:
print(results[1].name, ":", results[1].value)

estimation : 0.3699392589269321


## References

<a name='MCMF'>[1]</a>: [Paul Glasserman, Monte Carlo Methods in Financial Engineering. Springer-Verlag New York, 2003, p. 596.](https://link.springer.com/book/10.1007/978-0-387-21617-1)
<br><a name='AEA'>[2]</a>: [Gilles Brassard, Peter Hoyer, Michele Mosca, and Alain Tapp, Quantum Amplitude Amplification and Estimation. Contemporary Mathematics 305 (2002)](https://arxiv.org/abs/quant-ph/0005055)
<br><a name='QAR'>[3]</a>: [ Nikitas Stamatopoulos, Daniel J. Egger, Yue Sun, Christa Zoufal, Raban Iten, Ning Shen, and Stefan Woerner, Option Pricing using Quantum Computers, Quantum 4, 291 (2020).
](https://arxiv.org/abs/1905.02666v5)
